## SFTP通信で定期的にWebサーバから画像を取り出すプログラム

In [ ]:
import paramiko # 通信用ライブラリ
import schedule # 定期実行用ライブラリ
import time

In [ ]:
# カメラのIPアドレスの設定
# 接続するカメラの設定（6台分）
remote_param = []
remote_param.append({'HOST':'IPアドレス','PORT':22,'USER':'ユーザ名','PASS':'パスワード'})
remote_param.append({'HOST':'IPアドレス','PORT':22,'USER':'ユーザ名','PASS':'パスワード'})
remote_param.append({'HOST':'IPアドレス','PORT':22,'USER':'ユーザ名','PASS':'パスワード'})
remote_param.append({'HOST':'IPアドレス','PORT':22,'USER':'ユーザ名','PASS':'パスワード'})
remote_param.append({'HOST':'IPアドレス','PORT':22,'USER':'ユーザ名','PASS':'パスワード'})
remote_param.append({'HOST':'IPアドレス','PORT':22,'USER':'ユーザ名','PASS':'パスワード'})

# ダウンロード元と先の設定
remote_dir = '/***/***' # ダウンロード元PATH
download_dir = '***\\***\\'# ダウンロード先の共通フォルダPATH

In [23]:
# INIT関数（サーバ内データクリア）
# Webサーバ内の画像フォルダを空にする
def clear_server(HOST,PORT,SFTP_USER,SFTP_PASSWORD,REMOTE_DIR):
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    client.connect(HOST,port=PORT,username=SFTP_USER,password=SFTP_PASSWORD)
    
    sftp_connection = client.open_sftp()
    sftp_connection.chdir(REMOTE_DIR)
    print(sftp_connection.listdir())
    
    for _files in sftp_connection.listdir():
        if 'jpg' in _files: # jpgファイルを削除
            sftp_connection.remove(REMOTE_DIR+'/'+_files)
    sftp_connection.close()

In [26]:
# 設定したIPアドレスをSFTPで接続してクライアント情報を返す
def init_connect_sftp(HOST,PORT,SFTP_USER,SFTP_PASSWORD):
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    client.connect(HOST,port=PORT,username=SFTP_USER,password=SFTP_PASSWORD)
    
    return client

In [28]:
def sftp_download(HOST,PORT,SFTP_USER,SFTP_PASSWORD,REMOTE_DIR,DOWNLOAD_DIR,client):
    sftp_connection = client.open_sftp()
    sftp_connection.chdir(REMOTE_DIR)
    
    for _files in sftp_connection.listdir():
        if '.jpg' in _files:
            sftp_connection.get(_files,DOWNLOAD_DIR + HOST + '\\' +_files)
            sftp_connection.remove(REMOTE_DIR+'/'+_files)
    sftp_connection.close()

In [ ]:
sftp_client = []
for param in remote_param:
    clear_server(param['HOST'],param['PORT'],param['USER'],param['PASS'],remote_dir)# Webサーバ内jpgファイル削除
    sftp_client.append(init_connect_sftp(param['HOST'],param['PORT'],param['USER'],param['PASS'])) # 新たにSFTP通信の確立

In [29]:
# 定期実行するジョブの定義
def job():
    for param,client in zip(remote_param,sftp_client):
        sftp_download(param['HOST'],param['PORT'],param['USER'],param['PASS'],remote_dir,download_dir,client) # jpgファイルをダウンロード

In [30]:
# 指定した時間（分）でjob()を実行
schedule.every("時間を入れる").minutes.do(job)

Every 1 minute do job() (last run: [never], next run: 2020-07-06 15:43:33)

In [31]:
# 設定した定期実行ファイルを無限ループで動かす
while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 